In [0]:
!pip install dnspython

In [0]:
import dns
import pymongo
import pandas as pd

In [0]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb+srv://columbia:<password>@bigdataanalytics-d97sj.gcp.mongodb.net/test?retryWrites=true&w=majority")


In [0]:
# Read and upload data (ONLY PERFORM ONCE)
df = pd.read_csv("/content/drive/My Drive/Big Data Analytics/Data/fires_processed.csv")
records = df.to_dict('records')

In [0]:
len(records)

1880465

In [0]:
client['Fires'].fires.insert_many(records)

In [0]:
# Pull Dataset
db = client['Fires']
col = db['fires']

In [0]:
# Fixing the data with Unknown Causes
col.update_many( { "STAT_CAUSE_DESCR": {"$nin": ["Arson", "Children", "Lightning", "Debris Burning", "Miscellaneous", "Equipment Use", "Smoking", "Railroad"] } }, { "$set": { "STAT_CAUSE_DESCR": "Unknown" } })

OperationFailure: ignored

In [0]:
# Fixing the data with Unknown States
col.update_many( {"$expr": { "$gt": [ { "$strLenCP": "$STATE" }, 2 ] }} , { "$set": { "STATE": "Unknown" } })

**Sample Queries**

The following are sample queries used to test MongoDB before being ported into Flask web application.

In [0]:
myquery = { "STATE": "CA", "STAT_CAUSE_DESCR": "Debris Burning" }
doc = col.find(myquery)

In [0]:
col.distinct("STAT_CAUSE_DESCR")

['Miscellaneous',
 'Lightning',
 'Debris Burning',
 'Campfire',
 'Equipment Use',
 'Arson',
 'Children',
 'Railroad',
 'Smoking',
 '1.0',
 '7.0',
 '4.0',
 '5.0',
 '67',
 '2.0',
 '1307',
 '1300']

In [0]:
doc = col.aggregate([
    {"$group" : {"_id":"$STATE", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit": 10}
])

In [0]:
for x in doc:
  print(x)

{'_id': 'CA', 'count': 4747}
{'_id': 'AZ', 'count': 3558}
{'_id': 'OR', 'count': 3010}
{'_id': 'ID', 'count': 2432}
{'_id': 'MT', 'count': 2338}
{'_id': 'NM', 'count': 1592}
{'_id': 'CO', 'count': 919}
{'_id': 'UT', 'count': 876}
{'_id': 'WA', 'count': 787}
{'_id': 'MS', 'count': 574}
